# Detection of Malicicous Data

## Part I: Preprocessing 

### Read the data.file
The data file includes dataset structured like Basic Safety Messages (BSM) defined in the SEA J2735 standard, including vehicle's location, speed, acceleration, heading, and brake status. We utilize the data to check the correcteness of the data and better classify the five types of position spoofing attack. 

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

In [18]:
cols = [2,3,4,9,10,16] #id, xr ,yr, xt,yt, label
data = pd.read_csv("dataset/attack1withlabels.csv",usecols=cols)
data = data.dropna(axis=0, how="any")#remove invalid data
data_id = data.iloc[:,0]
# attack 1: distance is too far recevier and transmitter
pos_xr = data.iloc[:,1]
pos_yr = data.iloc[:,2]
pos_xt = data.iloc[:,3]
pos_yt = data.iloc[:,4]

label = data.iloc[:,5]


### colunm data
1. Type (3=BSM)
2. Time BSM was received by the receiver
3. Receiver ID
4. Receiver X position
5. Receiver Y position
6. Receiver Z position
7. Time BSM was transmitted
8. Transmitter ID
9. BSM ID
10. Transmitter X position
11. Transmitter Y position
12. Transmitter Z position
13. Transmitter X velocity
14. Transmitter Y velocity
15. Transmitter Z velocity
16. RSSI (Received Signal Strength Indicator)
17. Label (0=Normal Behavior)

## impletation keras

In [93]:
from tensorflow.python.keras import backend as k
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers.recurrent import LSTM
import tensorflow
from sklearn import model_selection


Using TensorFlow backend.


In [96]:
# model implementation
model = Sequential()
# model.add(LSTM(32, input_shape=(None,5),return_sequences=False))
model.add(Dense(1,input_dim=5))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [20]:
# We read the data from the data.
cols = [2,3,4,9,10,15,16] #id, xr ,yr, xt, yt, RSSI, label.
data = pd.read_csv("dataset/attack1withlabels.csv", usecols=cols, header=None)
X = data.iloc[:,:6]
y = data.iloc[:,6]

Two observations for attack 1
* The fake distance exceed the upper bound of communication range which is 800m
* The distance does not change but the RSSI changes.
The first criteria is trivial, thus we apply a filter. 

In [40]:
def distance(X):
    distance = np.zeros(len(X))
    for i in range(len(X)):
        distance[i] = np.linalg.norm([X.iloc[i][3]-X.iloc[i][9], X.iloc[i][4]-X.iloc[i][10]])
    distance_series = pd.Series(distance)
    return distance_series

distance_series = distance(X)
X['distance'] = distance_series

We filter the BSM whose distance is above the thredhold = 800

In [63]:
def filtering_dis(X,y,thredhold):
    drop_index = np.zeros(len(y))
    for i in range(len(y)):
        if X['distance'][i] > thredhold:
            drop_index[i]=1
    return drop_index

drop_index = filtering_dis(X,y,800)
drop_index_ = np.where(drop_index>0)
np.asarray(drop_index_)[0]
X_filter = X.drop(np.asarray(drop_index_)[0])
y_filter = y.drop(np.asarray(drop_index_)[0])

After pre-processing, we get the following relevant information:
1. The ID of the transmitter BSM (Coloum 2)
2. Transmitter positions (Coloum 9,10)
3. The distance between transmitter and receiver (Coloum 'distance')
4. The RSSI (Coloum 16)
We feed the following information into the training model.

In [109]:
sum(y_filter)/y_filter.shape[0]

0.13289369765437548

In [90]:
X_feed = X_filter.drop([9,10], axis = 1)

data_count = y_filter.count()
data_rate = 0.8 * data_count
data_rate = int(data_rate)

X_train = X_feed.iloc[:data_rate,:]
y_train = y_filter.iloc[:data_rate]
X_test = X_feed.iloc[data_rate:,:]
y_test = y_filter.iloc[data_rate:]

X_train.shape

(266402, 5)

In [97]:
#fit data
model.fit(X_train.values,y_train.values,epochs=100,batch_size=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
266402/266402 [==============================] - 4s 13us/step - loss: 2.2193 - acc: 0.8623
Epoch 2/100
266402/266402 [==============================] - 3s 13us/step - loss: 2.2192 - acc: 0.8623
Epoch 3/100
266402/266402 [==============================] - 3s 13us/step - loss: 2.2192 - acc: 0.8623
Epoch 4/100
266402/266402 [==============================] - 3s 13us/step - loss: 2.2192 - acc: 0.8623
Epoch 5/100
266402/266402 [==============================] - 3s 13us/step - loss: 2.2192 - acc: 0.8623: 0s - loss: 2.2184 - acc:
Epoch 6/100
266402/266402 [==============================] - 3s 13us/step - loss: 2.2192 - acc: 0.8623
Epoch 7/100
266402/266402 [==============================] - 3s 13us/step - loss: 2.2192 - acc: 0.8623
Epoch 8/100
266402/266402 [==============================] - 3s 13us/step - loss: 2.2192 - acc: 0.8623
Epoch 9/100
266402/266402 [==============================] - 3s 13us/step - loss: 2.2192 - acc: 0.8623

In [98]:
cost = model.evaluate(X_test.values,y_test.values,batch_size=100)
print('loss \n',cost[0])
print('accurency \n',cost[1])

66601/66601 [==============================] - 0s 6us/step
loss 
 1.8329824162138717
accurency 
 0.8862779837549901
